**写在前面:**, 这次新闻文本分类的比赛是由DataWhale与阿里天池联合举办，定位为nlp入门级赛事，[这里有详细的赛题说明](https://tianchi.aliyun.com/competition/entrance/531810/information)

## 问题置顶
- 怎么选择df某一行的内容并将其转换为string类型
- 

### 赛题理解
- 任务目标：文本分类（14类）
- 数据格式：每条样本由text和label组成；text进行了字符级的匿名处理（由数字表示原来的字符），label由0~13这14个数字组成，表示财经等14类文本。
- 评价标准：f1  越高越好
- 可选思路：TF-IDF+LR等传统分类模型；word2vec做特征+RNN+softmax；bert做特征+softmax分类

### task2_0722 数据读取与数据分析
- 查看训练集结构及规模
- 获取最大text长度
- 建立词典，获取词频

In [1]:
import pandas as pd

In [9]:
train_path = 'train_set.csv/train_set.csv'
train_set = pd.read_csv(train_path, encoding = 'utf-8')
train_set.head()

,label\ttext
0,2\t2967 6758 339 2021 1854 3731 4109 3792 4149...
1,11\t4464 486 6352 5619 2465 4802 1452 3137 577...
2,3\t7346 4068 5074 3747 5681 6093 1777 2226 735...
3,2\t7159 948 4866 2109 5520 2490 211 3956 5520 ...
4,3\t3646 3055 3055 2490 4659 6065 3370 5814 246...


In [13]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   label	text  200000 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


In [3]:
#看一下多少样本，每个样本几个属性
train_set.shape

(200000, 1)

In [22]:
#分别得到train_text 和 train_label

#train_set.loc[0]['label\ttext']
train_text, train_label = [],[]
for i in range(20000):
    tmp = train_set.loc[i]['label\ttext'].split('\t')
    a, b = tmp[1], tmp[0]
    train_label.append(b)
    train_text.append(a)
    
#将test list化
train_text_lst = []
for i in range(20000):
    train_text_lst.append(train_text[i].split())
len(train_text_lst)

20000

In [27]:
#获取最大的text长度
def get_len_text(text_list):
    Max = 0
    for i, it in enumerate(text_list):
        Max= len(it) if Max<len(it) else Max
    return Max
get_len_text(train_text_lst)

44665

In [31]:
#先把所有word都放到all_words里面
all_words = []
for i in train_text_lst:
    for j in i:
        all_words.append(j)
print("一共有"+str(len(all_words))+"个words.")

一共有18092357个words.


In [37]:
#建立词典 词典中的每一个item()是 word:[id, freq]
from collections import Counter
def build_vocab(all_words):
    cnt = Counter(all_words) #得到一个字典 word:freq
    res = {}
    for word,freq in cnt.items():
        res[word] = [len(res), freq]
    return res
print("词典中包含 " + str(len(build_vocab(all_words))) + " 个单词" )
voc = build_vocab(all_words)
                  
def id2word(id,voc):
    for word,lst in voc.items():
        if lst[0] == id:
            return word
def word2id(word,voc):
    return voc[word][0]
print(id2word(2,voc))
print(word2id('667',voc))

词典中包含 5697 个单词
339
2231
